# Pluvial Representative Subset of Events

__Description__:

__Input__: 

__Output__: 

---
## Load Libraries, Parameters, and File Paths:
### Libraries:

In [25]:
import os
import json
import numpy as np
import pandas as pd
import pathlib as pl
from matplotlib import pyplot as plt

### Parameters:
#### Site specific:

In [26]:
Project_Area = 'DC'

root_dir = pl.Path(os.getcwd())
inputs_dir = root_dir/f'Inputs/{Project_Area}'
outputs_dir = root_dir/f'Outputs/{Project_Area}'

#### Project specific (global):

In [27]:
binwidth = 0.01       # [inches]; increment for binning the excess rainfall amount
RI_max = 3000

verbose = True        # Option to display print statements
display_plots = True  # Option to display plots

### File Paths:
#### Weights:

In [28]:
weight_files = []
for f in inputs_dir.glob('*.json'):
    if 'Weights' in f.stem and 'TRI' not in f.stem: 
        weight_files.append(f)       
        print(f)

/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P03_D02_Weights.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P01_D01_Weights.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P04_D01_Weights.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P06_D01_Weights.json


#### Forcing:

In [29]:
forcing_files = []
for f in inputs_dir.glob('**/*.json'):
     if 'Weights' not in f.stem:
        forcing_files.append(f)
        print(f)

/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P04_D01.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P06_D01.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P03_D01.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P01_D01.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P01_D02.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P06_D02.json
/home/sputnam/gitrepos/pfra-hydromet/Inputs/DC/DC_P03_D02.json


---
## Calculate a Representative Subset of Events:

In [30]:
dic = {}
dic_vol = {}
for forcing_file in forcing_files:
    
    _, Pluvial_Model, Domain = forcing_file.stem.split('_')
    
    for w in weight_files:
        if Pluvial_Model in w.stem:
            weight_file = w
            
    with open(forcing_file) as f:
            ff_dic =  json.load(f)
    
    with open(weight_file) as f:
            wt =  json.load(f)  
            
    mainBCN = list(wt['BCName'].keys())[0]
    
    df = pd.DataFrame()
    df['Events'] = list(wt['BCName'][mainBCN].keys())
    df['Weight'] = list(wt['BCName'][mainBCN].values())    
    df = df.set_index('Events')
    for d in list(ff_dic.keys()):
        for k, v in ff_dic[d]['BCName'][Domain].items():
            df.loc[k, 'Volume'] = sum(v)
            df.loc[k, 'Dur'] = d
    df = df.reset_index().sort_values(by=['Volume'], ascending = True).set_index('Events') 
    
    binmin = min(df['Volume'])
    binmax = max(df['Volume'])
    bins = np.arange(binmin, binmax+binwidth, binwidth)
    
    lst = []
    for i, b in enumerate(bins[:]):
        if i==0:
            weight = sum(df[(0.0 == df['Volume'])]['Weight'])
        else:
            weight = sum(df[(bins[i-1]  < df['Volume']) & (df['Volume']<=b)]['Weight'])
        lst.append((b, weight))
    
    binned  = pd.DataFrame()
    binned['Volume'] = [i[0] for i in lst]
    binned['Weight'] = [i[1] for i in lst]
    binned['Cumulative'] = binned['Weight'].cumsum()
    binned['Theoretical RI'] = 1.0/(0.5-binned['Cumulative'])
    
    k = '{0}_{1}'.format(Pluvial_Model, Domain)
    dic[k] = {}
    dic_vol[k] = {}
    for e in df.index: 
        vol = df.loc[e,'Volume']
        dur = df.loc[e,'Dur']
        ename = '{0}_{1}'.format(dur, e)
        dic_vol[k][ename] = vol
        if vol==0:
            dic[k][ename] = min(round(binned.loc[0, 'Theoretical RI']), RI_max )
        #elif vol==df['Volume'].max():
        #    dic[k][ename] = binned.iloc[-2].loc['Theoretical RI']
        else:
            for j in binned.index[1:]:
                v0 = binned.loc[j-1, 'Volume']
                v1 = binned.loc[j, 'Volume']
                if v0<vol<=v1:
                    #dic[k][ename] = binned.loc[j, 'Theoretical RI']
                    dic[k][ename] = min(round(binned.loc[j, 'Theoretical RI']), RI_max ) 

    # TTRI = binned.loc[0]['Theoretical RI']
        
    # if verbose:
    #     print('{0} {1} Theoretical Threshold RI: {2} years'.format(Pluvial_Model, Domain, TTRI))
        
    # vavg = []
    # for i, b in enumerate(binned['Volume']):
    #     if i==0:
    #         vavg.append(b)
    #     elif i>0:
    #         vavg.append((b+binned['Volume'][i-1])/2)
    # binned['Vavg'] = vavg 
    
    # event = []
    # for vavg in binned['Vavg']:
    #     diff = df['Volume']-vavg
    #     event.append(diff[diff.abs().argsort()].index[0])
    # binned['Event'] = event    
    # binned.head()
    
    # unique_events = list(set(list(binned['Event'])))
    # edf = pd.DataFrame(data = {'Event': unique_events})
    # for i, e in enumerate(edf['Event']):
    #     edf.loc[i, 'Weight'] = sum(binned[binned['Event']==e]['Weight']) 
    #     edf.loc[i, 'Volume'] = df.loc[e]['Volume']
    # edf = edf[edf['Weight']>0.0].copy(deep=True)
    # assert np.round(sum(edf['Weight']), 2) == 0.5 , 'Total weight does not equal 0.5 as expected'
    # edf = edf.sort_values('Volume').reset_index(drop = True)
    # edf['Cumulative'] = edf['Weight'].cumsum()
    # print('Simulations to include in the Global Scale Test: {}'.format(edf.shape[0]))
    
    # fig, ax = plt.subplots(2, 1, figsize=(18, 12))
    # ax[0].plot(df['Weight'], df['Volume'], linestyle = '', marker = '.' , label = 'All Events', color = 'gray', alpha = 0.75)
    # ax[0].plot(edf['Weight'], edf['Volume'], linestyle = '', marker = '.', color = 'blue', label = 'Subset')
    # ax[0].set_xlabel('Weight, [-]', fontsize = 10)
    # ax[0].set_ylabel('Excess Rainfall, [inches]', fontsize=10)
    # ax[0].set_xlim([0, 0.002])
    # ax[0].legend()
    # ax[0].grid()    
    # ax[1].plot(binned['Volume'], 0.5 - binned['Cumulative'], linestyle = '-', marker = '', label = 'All Events', color = 'gray', alpha = 0.75)
    # ax[1].plot(edf['Volume'], 0.5-edf['Cumulative'], linestyle = '-', marker = '', color = 'blue', label = 'Subset')
    # ax[1].set_xlabel('Excess Rainfall, [inches]', fontsize = 10)
    # ax[1].set_ylabel('Weight (~ probability)', fontsize=10)
    # ax[1].legend()
    # ax[1].grid()       
    
    # edic = {'BCName':{mainBCN:{}}}
    # elist =  []
    # for i, e in enumerate(edf['Event']):
    #     edic['BCName'][mainBCN][e] = edf.loc[i]['Weight']
    #     elist.append(e)
    # with open(outputs_dir/'{0}_{1}_{2}_Weights_TRI.json'.format(Project_Area, Pluvial_Model, Domain), 'w') as f:
    #     json.dump(edic, f)
        
    # df = pd.DataFrame(data = {'Events': elist})
    # df = df.set_index('Events')
    # df.to_csv(outputs_dir/'{0}_{1}_{2}_Events_TRI.csv'.format(Project_Area, Pluvial_Model, Domain))    

In [31]:
writer = pd.ExcelWriter(outputs_dir/'{0}_Pluvial_TRI.xlsx'.format(Project_Area))
for k in dic.keys():
    tempdf = pd.DataFrame.from_dict(dic[k], orient='index', columns=['TRI'])
    tempdf.index.name = 'Events'
    for e in tempdf.index:
        tempdf.loc[e, 'Volume'] = dic_vol[k][e]
    tempdf['Dur'] = [x.split('_')[0] for x in tempdf.index]
    tempdf = tempdf[tempdf['Dur']=='H24'].copy(deep=True)    
    tempdf.sort_values('Volume', inplace = True)
    tempdf.drop(columns=['Dur']).to_excel(writer, sheet_name = k)
writer.save()    

    